## tf.custom_gradient

@Date    : Nov-29-22 15:44:54

@Author  : Kan Huang (kan.huang@connect.ust.hk)

@RefLink : https://www.tensorflow.org/api_docs/python/tf/custom_gradient

@RefLink : https://stackoverflow.com/questions/56657993/how-to-create-a-keras-layer-with-a-custom-gradient-in-tf2-0/56658149

In [44]:
import numpy as np
import tensorflow as tf
tf.__version__

'2.3.0'

### Single variable function

In [45]:
def log1pexp(x):
    return tf.math.log(1 + tf.exp(x))


x = tf.constant(100.)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = log1pexp(x)

grad = tape.gradient(y, x)  # Will be NaN when evaluated.

In [46]:
print(grad)

tf.Tensor(nan, shape=(), dtype=float32)


In [47]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)

    def grad(upstream):
        return upstream * (1 - 1 / (1 + e))

    return tf.math.log(1 + e), grad

x = tf.constant(100.)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = log1pexp(x)

grad = tape.gradient(y, x)

In [48]:
print(grad)

tf.Tensor(1.0, shape=(), dtype=float32)


### Applied to a layer

In [49]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(CustomLayer, self).__init__()

    def call(self, x):
        # you don't need to explicitly define the custom gradient
        # as long as you registered it with the previous method
        return log1pexp(x)


In [43]:
l = CustomLayer()

### Multiple variables function